# Z3 solver
https://github.com/Z3Prover/z3

Instalacija:  
`sudo apt-get install z3`  
`pip install z3-solver`

In [1]:
from z3 import *

In [14]:
# U iskaznoj logici zapisati uslov da je 4-bitna reprezentacija broja
# palindrom, ali da nisu svi bitovi isti.

A,B,C,D = Bools('A B C D')

s = Solver()
s.add(A == D, B == C, Not(And(A == B, B == C, C == D)))

if s.check() == sat:
    print(s.model())

[A = True, D = True, B = False, C = False]
[A = False, D = False, B = True, C = True]


In [ ]:
A,B,C,D = Bools('A B C D')

s = Solver()
s.add(A == D, B == C, Not(And(A == B, B == C, C == D)))

while s.check() == sat:
    print(s.model())
    # Ispis svih modela. U ogranicenje dodamo negaciju prethodnog resenja.
    # s.model()[A] -> vrednost promenljive A u poslednjem modelu koji zadovoljava ogranicenja
    s.add(Not(And(A == s.model()[A], B == s.model()[B], C == s.model()[C], D == s.model()[D])))

In [12]:
x, y, z = Reals('x y z')

s = Solver()
s.add(
    x + 5*y - 3*z == 4, # 1/2 +  7/2 = 8/2 = 4
    -x + y + z == 3,
    2*x + y - z == 1
)

if s.check() == sat:
    print(s.model())

[y = 7/4, x = 1/2, z = 7/4]


7/4

In [12]:
x, y, z = Reals('x y z')
s = Solver()
s.add(x > 1, y > 1, x + y > 3, z - x < 10)
print(s.check())

m = s.model()
print("x = %s" % m[x])

print("traversing model...")
for d in m.decls():
    print("%s = %s" % (d.name(), m[d]))

sat
x = 3/2
traversing model...
y = 2
x = 3/2
z = 0


# Uprošćavanje izraza

In [13]:
x, y = Reals('x y')
solve(x + 10000000000000000000000 == y, y > 20000000000000000)

print(Sqrt(2) + Sqrt(3))
print(simplify(Sqrt(2) + Sqrt(3)))
print(simplify(Sqrt(2) + Sqrt(3)).sexpr())
# The sexpr() method is available for any Z3 expression
print((x + Sqrt(y) * 2).sexpr())



[y = 20000000000000001, x = -9999979999999999999999]
2**(1/2) + 3**(1/2)
3.1462643699?
(root-obj (+ (^ x 4) (* (- 10) (^ x 2)) 1) 4)
(+ x (* (^ y (/ 1.0 2.0)) 2.0))


In [ ]:
x = Int('x')
y = Int('y')
print(simplify(x + y + 2*x + 3))
print(simplify(x < y + x + 2))
print(simplify(And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5))) 



In [23]:
#Bool p_i_j - Na polju (i,j) se nalazi dama
#Int Q_{i} -> Broj kolone u kojoj se nalazi dama u i-tom redu
# Q_4 = 3 -> U cetvrtom redu, dama se nalazi na trecoj koloni

# Q_3 = 2, Q_4 = 2

n = 8
Q = [Int(f'Q_{i}') for i in range(n)]

val_c = [And(0 <= q, q < n) for q in Q]
col_c = [Distinct(Q)]
diag_c = [
    And(Q[i] - Q[j] != i - j, Q[i] - Q[j] != j - i) 
    for i in range(n) for j in range(i) if i != j
]

n_queens = val_c + col_c + diag_c

solve(n_queens)

[Q_3 = 6,
 Q_1 = 1,
 Q_7 = 0,
 Q_5 = 7,
 Q_4 = 2,
 Q_0 = 4,
 Q_2 = 3,
 Q_6 = 5]


# Sudoku
za samostalan rad

In [ ]:
from z3 import *


X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(9) ] 
      for i in range(9) ]


cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9) 
             for i in range(9) for j in range(9) ]


rows_c   = [ Distinct(X[i]) for i in range(9) ]


cols_c   = [ Distinct([ X[i][j] for i in range(9) ]) 
             for j in range(9) ]

sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j] 
                        for i in range(3) for j in range(3) ]) 
             for i0 in range(3) for j0 in range(3) ]

sudoku_c = cells_c + rows_c + cols_c + sq_c

instance = ((0,0,0,0,9,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))

instance_c = [ If(instance[i][j] == 0, 
                  True, 
                  X[i][j] == instance[i][j]) 
               for i in range(9) for j in range(9) ]

s = Solver()
s.add(sudoku_c + instance_c)
if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ] 
          for i in range(9) ]
    print_matrix(r)
else:
    print("failed to solve")

# Funkcije 

In [15]:
x = Int('x')
y = Int('y')
f = Function('f', IntSort(), IntSort())
solve(f(f(x)) == x, f(x) == y, x != y)

[x = 0, y = 1, f = [1 -> 0, else -> 1]]


In [16]:
x = Int('x')
y = Int('y')
f = Function('f', IntSort(), IntSort())
s = Solver()
s.add(f(f(x)) == x, f(x) == y, x != y)
print(s.check())
m = s.model()
print("f(f(x)) =", m.evaluate(f(f(x))))
print("f(x)    =", m.evaluate(f(x)))

sat
f(f(x)) = 0
f(x)    = 1


# Predikatska logika

In [9]:
# all humans are mortal
# Socrates is a human
# so Socrates mortal 
############################################

from z3 import *

Object = DeclareSort('Object')

Human = Function('Human', Object, BoolSort())
Mortal = Function('Mortal', Object, BoolSort())

# a well known philosopher 
socrates = Const('socrates', Object)

# free variables used in forall must be declared Const in python
x = Const('x', Object)

axioms = [ForAll([x], Implies(Human(x), Mortal(x))), 
          Human(socrates)]


solver = Solver()
solver.add(axioms)

if solver.check() == sat:
    conjecture = Mortal(socrates)
    solver.add(Not(conjecture))
    if solver.check() == unsat:
        print('Conjecture proved!')
else:
    print('contradicting axioms')
    
# classical refutation


Conjecture proved!


In [7]:
# Dve nemimoilazne prave se seku ili su paralelne.
# Prave koje se seku leže u istoj ravni.
# Prave koje su paralelene leže u istoj ravni.
# Dve nemimoilazne prave leže u istoj ravni.

# m(X,Y) - X i Y su nemimoilazne: m: PxP -> B
# s(X,Y) - X i Y se seku: s: PxP -> B
# p(X,Y) - X i Y su paralelne: p: PxP -> B
# r(X,Y) - X i Y leze u istoj ravni: r: PxP -> B

B = BoolSort()
P = DeclareSort('Prave')
m = Function('m', P, P, B)
s = Function('s', P, P, B)
p = Function('p', P, P, B)
r = Function('r', P, P, B)

x, y = Consts('x y', P)

solver = Solver()
axioms = [
    ForAll([x,y], Implies(m(x,y), Or(s(x,y), p(x,y)))),
    ForAll([x,y], Implies(s(x,y), r(x,y))),
    ForAll([x,y], Implies(p(x,y), r(x,y)))
]

solver.add(axioms)

if solver.check() == sat:
    conjecture = ForAll([x,y], Implies(m(x,y), r(x,y)))
    solver.add(Not(conjecture))
    if solver.check() == unsat:
        print('Conjecture proved!')
else:
    print('contradicting axioms')

Conjecture proved!


In [11]:
# Svaka dva brata imaju zajedničkog roditelja.
# Roditelj je stariji od deteta.
# Postoje braća.
# Ni jedna osoba nije starija od druge.

# b: OxO -> B

B = BoolSort()
O = DeclareSort('Osoba')

b = Function('braca', O, O, B)
r = Function('roditelj', O, O, B)
s = Function('stariji', O, O, B)

x,y,z = Consts('x y z', O)

solver = Solver()
axioms = [
    ForAll([x,y], Exists([z], Implies(b(x,y), And(r(z,x), r(z,y))))),
    ForAll([x,y], Implies(r(x,y), s(x,y))),
    Exists([x,y], b(x,y))
]

if solver.check() == sat:
    conjecture = ForAll([x,y], Not(s(x,y)))
    solver.add(Not(conjecture))
    if solver.check() == unsat:
        print('Conjecture proved!')
else:
    print('contradicting axioms')